In [82]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from textblob import TextBlob
from itertools import chain
from collections import Counter

In [83]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [84]:
beeradvo = pd.read_csv('beeradvocate.csv') 

In [52]:
beeradvo

,mnf,name,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link
0,four winds brewing co.,nectarous,wild ale,5.50%,123,4.45,['look: 4.5 | smell: 4.75 | taste: 4.25 | feel...,"['4.36', '4.2', '5', '3.4', '3.9', '4.54', '4....",35,"[""Poured from the can into a teku glass. No vi...",https://www.beeradvocate.com/beer/profile/3218...
1,driftwood brewery,sartori harvest ipa,ipa - american,7.00%,117,4.4,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.28', '4.3', '4.12', '4.24', '4.41', '4.32'...",42,"[""I've heard great things...now I've tasted th...",https://www.beeradvocate.com/beer/profile/1885...
2,driftwood brewery,fat tug ipa,ipa - american,7.00%,718,4.27,['look: 3.75 | smell: 4.25 | taste: 4 | feel: ...,"['4.05', '4.31', '4.81', '4.17', '3.94', '3.88...",184,['Superb lacing & pours golden pale in the gla...,https://www.beeradvocate.com/beer/profile/1885...
3,central city brewers + distillers,red racer ipa (india pale ale),ipa - american,6.50%,1136,4.25,['look: 4 | smell: 3 | taste: 4 | feel: 3 | ov...,"['3.46', '4.21', '3.86', '4.75', '4.5', '3.71'...",499,['Look: Clear dark copper color. Pours a light...,https://www.beeradvocate.com/beer/profile/1032...
4,driftwood brewery,singularity,stout - russian imperial,10.60%,180,4.28,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.43', '4.87', '3.2', '4.1', '3.83', '4.21',...",70,"[""2oz bottle (2012, label says 14.0% ABV as op...",https://www.beeradvocate.com/beer/profile/1885...
5,driftwood brewery,raised by wolves ipa,ipa - new england,7.00%,79,4.32,['look: 5 | smell: 4.75 | taste: 4.5 | feel: 5...,"['4.69', '3', '4.6', '4.17', '4.24', '4.46', '...",33,['Smells like one of those bags of dried fruit...,https://www.beeradvocate.com/beer/profile/1885...
6,driftwood brewery,naughty hildegard esb,bitter - english extra special / strong bitter...,6.50%,140,4.25,['look: 4 | smell: 3 | taste: 3 | feel: 4 | ov...,"['3.16', '4.21', '4', '4.29', '4.1', '3.61', '...",47,"[""Clear dark amber color. The head was a light...",https://www.beeradvocate.com/beer/profile/1885...
7,four winds brewing co.,juxtapose wild ipa,ipa - american,6.50%,133,4.2,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.4', '4.19', '3.99', '2.6', '4.13', '4.13',...",41,"[""This is new to me. Poured from a 1 pint can,...",https://www.beeradvocate.com/beer/profile/3218...
8,central city brewers + distillers,red racer imperial ipa,ipa - imperial,9.00%,190,4.17,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.23', '3.26', '3.08', '4.15', '3.86', '3.4'...",66,"[""Appearance: Clear medium copper, decent carb...",https://www.beeradvocate.com/beer/profile/1032...
9,driftwood brewery,latus,sour - flanders red ale,7.50%,53,4.3,"['SebD from Canada', 'look: 4 | smell: 4.25 | ...","['4.75', '4.47', '3.89', '4.21', '3.71', '4.46...",18,['Big thanks to FondueVoodoo for helping me ro...,https://www.beeradvocate.com/beer/profile/1885...


In [ ]:
beeradvo.info()

### Clean dataset

In [85]:
beeradvo.name = beeradvo.name.str.lower()
beeradvo.mnf = beeradvo.mnf.str.lower()
beeradvo.style = beeradvo['style'].str.lower()

In [46]:
text_reviews = beeradvo.text_reviews.str.lower()

In [57]:
text_reviews.shape

(513,)

#### 1. Remove Punctuation 

In [ ]:
#remove punctuation
def remove_punc(text):
    no_punct = "".join([i for i in text if i not in string.punctuation])
    return no_punct

In [ ]:
text_reviews_cleaned = text_reviews.apply(remove_punc)

In [ ]:
text_reviews_cleaned.shape

In [ ]:
# find individual review
indi_review = text_reviews_cleaned.str.split("[\w]{3}\s[\d]{2}\s[\d]{4}",expand=True).T

In [ ]:
indi_review.head(2)

In [ ]:
#indi_review.to_csv("indi_review.csv")

In [58]:
indi_review = pd.read_csv("indi_review.csv",dtype=str).drop('Unnamed: 0',axis=1)

In [59]:
indi_review.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
0,poured from the can into a teku glass no visib...,ive heard great thingsnow ive tasted them serv...,superb lacing pours golden pale in the glass ...,look clear dark copper color pours a light hea...,2oz bottle 2012 label says 140 abv as opposed ...,smells like one of those bags of dried fruit t...,clear dark amber color the head was a light ta...,this is new to me poured from a 1 pint can it ...,appearance clear medium copper decent carbonat...,big thanks to fonduevoodoo for helping me roun...,light golden colour that pops bright tropical ...,novem,bottle cellared since 2016 deep mahogany pour ...,0 ml bottle split two ways cool label a slight...,hazy orangetan color not a lot of head and it ...,oz bomber split 2 ways deep golden pour with ...,crystal clear deep golden honey pour with fing...,clear dark orangetan color formed a thick head...,such a classic almost taste the land where thi...,bottle from tres bonne cuisine darkinky black ...,bomber from the lcbo best before,cloudy goldtan color the head was a mix of bub...,pours crystal clear with a lively white head a...,solid ipa right here beautiful dark golden col...,served on tap at royal dinette pours a deeply ...,opaque black dark brown only in thin sections ...,50ml corked and caged bottle poured tulip a wa...,bought a 6pk at the bc liquor store in victori...,looks like oj with a little head hop aroma exp...,can pours orange and hazy incredible grapefrui...,bomber sa sensory pinta blackish brown with ...,its a lovely brew i wish this beer was availa...,sours arent my favorite style but theyre growi...,poured from a 473ml can the 2017 run is 52 abv...,murky orangebrown color many smallmedium bubbl...,just tried it for the first time vaulted into...,cellar diving in early fall 2019 and coming up...,copper filtered with moderate carbonation and ...,copper filtered with moderate carbonation in f...,beautiful deep brown with thick creamy head lo...,superflux beer co rainbow machine ipa 75 ser...,well this is 15 months past its best by date,this is a review for the citr

In [60]:
indi_review.shape

(3146, 513)

#### 2. Tokenize

In [ ]:
indi_review1 = indi_review.fillna('No review',inplace=False)
indi_review1 = indi_review1.replace('','No review')

In [ ]:
review_token = pd.DataFrame()

In [ ]:
for i in indi_review.columns:
    review_token[i] = indi_review1[i].apply(nltk.word_tokenize)

In [ ]:
review_token[0]

In [ ]:
#review_token.to_csv("review_token.csv")

In [65]:
review_token = pd.read_csv("review_token.csv",dtype=str).drop('Unnamed: 0',axis=1)

In [66]:
review_token.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
0,"['poured', 'from', 'the', 'can', 'into', 'a', ...","['ive', 'heard', 'great', 'thingsnow', 'ive', ...","['superb', 'lacing', 'pours', 'golden', 'pale'...","['look', 'clear', 'dark', 'copper', 'color', '...","['2oz', 'bottle', '2012', 'label', 'says', '14...","['smells', 'like', 'one', 'of', 'those', 'bags...","['clear', 'dark', 'amber', 'color', 'the', 'he...","['this', 'is', 'new', 'to', 'me', 'poured', 'f...","['appearance', 'clear', 'medium', 'copper', 'd...","['big', 'thanks', 'to', 'fonduevoodoo', 'for',...","['light', 'golden', 'colour', 'that', 'pops', ...",['novem'],"['bottle', 'cellared', 'since', '2016', 'deep'...","['0', 'ml', 'bottle', 'split', 'two', 'ways', ...","['hazy', 'orangetan', 'color', 'not', 'a', 'lo...","['oz', 'bomber', 'split', '2', 'ways', 'deep',...","['crystal', 'clear', 'deep', 'golden', 'honey'...","['clear', 'dark', 'orangetan', 'color', 'forme...","['such', 'a', 'classic', 'almost', 'taste', 't...","['bottle', 'from', 'tres', 'bonne', 'cuisine',...","['bomber', 'from', 'the', 'lcbo', 'best', 'bef...","['cloudy', 'goldtan', 'color', 'the', 'head', ...","['pours', 'crystal', 'clear', 'with', 'a', 'li...","['solid', 'ipa', 'right', 'here', 'beautiful',...","['served', 'on', 'tap', 'at', 'royal', 'dinett...","['opaque', 'black', 'dark', 'brown', 'only', '...","['50ml', 'corked', 'and', 'caged', 'bottle', '...","['bought', 'a', '6pk', 'at', 'the', 'bc', 'liq...","['looks', 'like', 'oj', 'with', 'a', 'little',...","['can', 'pours', 'orange', 'and', 'hazy', 'inc...","['bomber', 'sa', 'sensory', 'pinta', 'blackish...","['its', 'a', 'lovely', 'brew', 'i', 'wish', 't...","['sours', 'arent', 'my', 'favorite', 'style', ...","['poured', 'from', 'a', '473ml', 'can', 'the',...","['murky', 'orangebrown', 'color', 'many', 'sma...","['just', 'tried', 'it', 'for', 'the', 'first',...","['cellar', 'diving', 'in', 'early', 'fall', '2...","['copper', 'filtered', 'with', 'moderate', 'ca...","['copper', 'filtered', 'with', 'moderate', 'ca...","['beautiful', 'deep', 'brown', 'with', 'thick'...","['superflux', 'b

In [67]:
review_token.shape

(3146, 513)

#### 3. Remove stopwords

In [ ]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [ ]:
no_stopwords = pd.DataFrame()

In [ ]:
for i in review_token.columns:
    no_stopwords[i] = review_token[i].apply(lambda x: remove_stopwords(x))

In [ ]:
no_stopwords.columns

In [ ]:
#no_stopwords.to_csv("no_stopwords.csv")

In [68]:
no_stopwords = pd.read_csv("no_stopwords.csv").drop('Unnamed: 0',axis=1)

In [6]:
no_stopwords.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
0,"['poured', 'teku', 'glass', 'visible', 'dating...","['ive', 'heard', 'great', 'thingsnow', 'ive', ...","['superb', 'lacing', 'pours', 'golden', 'pale'...","['look', 'clear', 'dark', 'copper', 'color', '...","['2oz', 'bottle', '2012', 'label', 'says', '14...","['smells', 'like', 'one', 'bags', 'dried', 'fr...","['clear', 'dark', 'amber', 'color', 'head', 'l...","['new', 'poured', '1', 'pint', 'immediate', 't...","['appearance', 'clear', 'medium', 'copper', 'd...","['big', 'thanks', 'fonduevoodoo', 'helping', '...","['light', 'golden', 'colour', 'pops', 'bright'...",['novem'],"['bottle', 'cellared', 'since', '2016', 'deep'...","['0', 'ml', 'bottle', 'split', 'two', 'ways', ...","['hazy', 'orangetan', 'color', 'lot', 'head', ...","['oz', 'bomber', 'split', '2', 'ways', 'deep',...","['crystal', 'clear', 'deep', 'golden', 'honey'...","['clear', 'dark', 'orangetan', 'color', 'forme...","['classic', 'almost', 'taste', 'land', 'farm',...","['bottle', 'tres', 'bonne', 'cuisine', 'darkin...","['bomber', 'lcbo', 'best']","['cloudy', 'goldtan', 'color', 'head', 'mix', ...","['pours', 'crystal', 'clear', 'lively', 'white...","['solid', 'ipa', 'right', 'beautiful', 'dark',...","['served', 'tap', 'royal', 'dinette', 'pours',...","['opaque', 'black', 'dark', 'brown', 'thin', '...","['50ml', 'corked', 'caged', 'bottle', 'poured'...","['bought', '6pk', 'bc', 'liquor', 'store', 'vi...","['looks', 'like', 'oj', 'little', 'head', 'hop...","['pours', 'orange', 'hazy', 'incredible', 'gra...","['bomber', 'sa', 'sensory', 'pinta', 'blackish...","['lovely', 'brew', 'wish', 'beer', 'available'...","['sours', 'arent', 'favorite', 'style', 'theyr...","['poured', '473ml', '2017', 'run', '52', 'abv'...","['murky', 'orangebrown', 'color', 'many', 'sma...","['tried', 'first', 'time', 'vaulted', 'top', '...","['cellar', 'diving', 'early', 'fall', '2019', ...","['copper', 'filtered', 'moderate', 'carbonatio...","['copper', 'filtered', 'moderate', 'carbonatio...","['beautiful', 'deep', 'brown', 'thick', 'cream...","['superflux', 'beer', 'co', 'rainbow', 

In [69]:
no_stopwords.shape

(3146, 513)

#### 4. Lemmatization

In [91]:
cleaned_no_stopwords = pd.DataFrame()

In [92]:
for i in no_stopwords.columns:
    cleaned_no_stopwords[i] = no_stopwords[i].apply(lambda x: x.replace('\'','').replace('[','').replace(']','').split(', '))

In [93]:
cleaned_no_stopwords.shape

(3146, 513)

In [94]:
lemmatizer = WordNetLemmatizer()

In [95]:
lemme_df = pd.DataFrame()

In [96]:
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag

tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [97]:
# first draft
def lemma_text(text):
    lemma = []
    for token, tag in pos_tag(text):
        lemma1 = lemmatizer.lemmatize(token,tag_map[tag[0]])
        lemma.append(lemma1)
    return lemma

In [98]:
lemma_df = pd.DataFrame()

In [103]:
for i in cleaned_no_stopwords.columns:
    try:
        lemma_df[i] = cleaned_no_stopwords[i].apply(lambda x: lemma_text(x))
    except IndexError:
        lemma_df[i] = 'None'

In [105]:
#lemma_df.to_csv("lemma_df.csv")

In [106]:
lemma_df = pd.read_csv("lemma_df.csv").drop("Unnamed: 0", axis=1)

In [73]:
lemma_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,425,426,428,429,430,431,433,434,435,436,437,438,440,441,442,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,504,505,506,507,509,510,511,512
0,"['pour', 'teku', 'glass', 'visible', 'date', '...","['ive', 'heard', 'great', 'thingsnow', 'ive', ...","['superb', 'lace', 'pours', 'golden', 'pale', ...","['look', 'clear', 'dark', 'copper', 'color', '...","['2oz', 'bottle', '2012', 'label', 'say', '140...","['smell', 'like', 'one', 'bag', 'dry', 'fruit'...","['clear', 'dark', 'amber', 'color', 'head', 'l...","['new', 'pour', '1', 'pint', 'immediate', 'two...","['appearance', 'clear', 'medium', 'copper', 'd...","['big', 'thanks', 'fonduevoodoo', 'help', 'rou...","['light', 'golden', 'colour', 'pop', 'bright',...",['novem'],"['bottle', 'cellared', 'since', '2016', 'deep'...","['0', 'ml', 'bottle', 'split', 'two', 'way', '...","['hazy', 'orangetan', 'color', 'lot', 'head', ...","['oz', 'bomber', 'split', '2', 'way', 'deep', ...","['crystal', 'clear', 'deep', 'golden', 'honey'...","['clear', 'dark', 'orangetan', 'color', 'form'...","['classic', 'almost', 'taste', 'land', 'farm',...","['bottle', 'tres', 'bonne', 'cuisine', 'darkin...","['bomber', 'lcbo', 'best']","['cloudy', 'goldtan', 'color', 'head', 'mix', ...","['pours', 'crystal', 'clear', 'lively', 'white...","['solid', 'ipa', 'right', 'beautiful', 'dark',...","['serve', 'tap', 'royal', 'dinette', 'pour', '...","['opaque', 'black', 'dark', 'brown', 'thin', '...","['50ml', 'cork', 'cage', 'bottle', 'poured', '...","['buy', '6pk', 'bc', 'liquor', 'store', 'victo...","['look', 'like', 'oj', 'little', 'head', 'hop'...","['pours', 'orange', 'hazy', 'incredible', 'gra...","['bomber', 'sa', 'sensory', 'pinta', 'blackish...","['lovely', 'brew', 'wish', 'beer', 'available'...","['sour', 'arent', 'favorite', 'style', 'theyre...","['pour', '473ml', '2017', 'run', '52', 'abv', ...","['murky', 'orangebrown', 'color', 'many', 'sma...","['try', 'first', 'time', 'vault', 'top', '4', ...","['cellar', 'dive', 'early', 'fall', '2019', 'c...","['copper', 'filter', 'moderate', 'carbonation'...","['copper', 'filter', 'moderate', 'carbonation'...","['beautiful', 'deep', 'brown', 'thick', 'cream...","['superflux', 'beer', 'co', 'rainbow', 'machin...","['well', '15', 'month',

In [107]:
lemma_df.shape

(3146, 513)

#### 5. Split long words

In [108]:
# there are long words that were joined by 2 single words (by typing error)
import wordninja
def split_long_word(text):
    return [i for i in wordninja.split(text) if i != "'" and len(i) > 2]

In [109]:
split_long_words = pd.DataFrame()

In [112]:
for i in lemma_df.columns:
    split_long_words[i] = lemma_df[i].apply(lambda x: split_long_word(x))

In [113]:
#split_long_words.to_csv("split_long_words.csv")

In [114]:
split_long_words = pd.read_csv("split_long_words.csv").drop("Unnamed: 0", axis=1)

In [115]:
split_long_words.shape

(3146, 513)

#### 6. Categorize taste and aroma

##### 6.1. Define elements for evaluation

In [6]:
# Define elements for evaluation

#TASTE (no umami and fat as it's not relevant to beer)
sweet = "|".join(('malt','sweet','syrup','sugar','carbo','luscious','candy','candied','cloy','honey','necta','saccharine'))
salty = "|".join(('salty','briny','saline','brackish'))
sour = "|".join(('sour','acid','tart','acetic','vinegar','biting'))
bitter = "|".join(('hop','bitter','harsh','acrid','astringent','acrimonious','expresso','roast','burn','smok',
                  'coffee','choco','char'))

#AROMA (only single word, no unnecessary aroma)
#https://www.flickr.com/photos/cizauskas/35617138930/in/album-72157671634412411/
floral = "|".join(('floral','flower','geranium','rose','rosé','lavender','lilac','violet','hawthorn',
                  'jasmine','lily','hyacinth','iris','citronella','perfume','botanical','hibicus'))
sweet_aromatic = "|".join(('sweet','sugar','bubblegum','marshmallow','frosting','honey',
                         'toffee','caramel','vanilla','chocolate'))
spicy = "|".join(('spic','licorice','anise','cinnamon','clove','pepper','coriander','lemongrass',
                'cumin','caraway','fenugreek','ginger','cardamom','chili','fennel','peel',
                 'peppercorn','peppermint','rooibos','tumeric'))
herbacious = "|".join(('herb','dill','thyme','rosemary','wintergreen','mint','tea','root','seed','gruit',
                      'fenugreek','ginseng','rooibos','rosehip','stevia','yerba','geranium'))
grassy = "|".join(('grass','plant','leave','leaf','hay'))
woody = "|".join(('wood','pine','cedar','eucalyptus','sage','rosewood','sandalwood','bark',
                'balsamic','resin','turpentine','camphor','tobacco','oak','tann','juniper','spruce'))
nutty = "|".join(('nut','almond','walnut','marzipan'))
grain = "|".join(('malt','grain','popcorn','toast','husk','biscuit','bread','cereal','cracker','pasta','polenta'))
burnt = "|".join(('burn','coffee','molasses','smok','char','roast','choco'))
vegetal = "|".join(('vege','bean','cucumber','celery','carrot','pea','cabbage',
                  'potato','corn','onion','leek','garlic','tomato','seaweed'))
alcoholic = "|".join(('alcohol','wine','rum','sherry','vinous','menthol'))
fruity =  "|".join(('fruit','apple','cider','pear','cherry','apricot','peach','cranberry','melon',
                  'watermelon','cantaloupe','honeydew','citrus','grapefruit','date','plum',
                  'lemon','lime','orange','berry','strawberry','raspberry','tangerine','citrus',
                  'rhubarb','grape','muscat','currant','coconut','tropical','yuzu', 'jam','prune',
                  'banana','pineapple','mango','papaya','pawpaw','guava','lychee','kiwi','juic','rosehip'))

In [7]:
#put those elements in their own list of objects for easy iteration
taste = [sweet,salty,sour,bitter]
aroma = [floral,sweet_aromatic,spicy,herbacious,grassy,woody,nutty,grain,burnt,vegetal,alcoholic,fruity]

In [8]:
#put those elements in their own list of objects for easy iteration
taste_dict = {'Sweet Taste':sweet,'Salty':salty, 'Sour':sour, 'Bitter':bitter}
aroma_dict = {'Floral':floral, 'Sweet Aroma': sweet_aromatic, 'Spicy': spicy,
         'Herbacious': herbacious, 'Grassy': grassy, 'Woody': woody,
         'Nutty': nutty,'Grain': grain, 'Burnt':burnt, 'Vegetal': vegetal,
         'Alcoholic': alcoholic, 'Fruity': fruity}

In [9]:
def categorize(text,mydict):
    m = [key for key,value in mydict.items() if re.search(value,text)]
    if m == []: return [None]
    else: return m

In [116]:
#Categorize Taste and Aroma
taste_categorized = pd.DataFrame()
aroma_categorized = pd.DataFrame()

for i in split_long_words.columns:
    m = split_long_words[i].apply(lambda x: categorize(x,taste_dict))
    n = split_long_words[i].apply(lambda x: categorize(x,aroma_dict))
    taste_categorized[i] = pd.Series(Counter(chain.from_iterable(m)))
    aroma_categorized[i] = pd.Series(Counter(chain.from_iterable(n)))

In [117]:
taste_categorized

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
Sweet Taste,28,35,151,425,55,20,42,29,57,14,4.0,54,39,13,23,8,29,22,19.0,67,35,8,10,16,7,12,2.0,32,3,2.0,18,23,7,7,5,3,36,84,14,5.0,1,12,2,11,8,1.0,16,3,4,47,43,59,3.0,1,33,26,14,6,14,3,30,13,NaN,21,12,1,43,5.0,56,3,10,6,22.0,1,10,30,1,5.0,2.0,6.0,5.0,5,5.0,13,21,3,25,26,8.0,3,8.0,3.0,23,10,13,2.0,12,11,93,6.0,5,3.0,3,2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1.0,7,NaN,3,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,3,5,3.0,NaN,1.0,5.0,NaN,4,1.0,6,3.0,NaN,2.0,NaN,31,29,2.0,1.0,NaN,4.0,7,NaN,NaN,73,4.0,4.0,5,8.0,1.0,3,5,6,13.0,8,1.0,3.0,3.0,1.0,NaN,151,20,NaN,8,5,1.0,7,4,NaN,1.0,NaN,29,NaN,28,5.0,1,4,4,8,NaN,NaN,NaN,NaN,NaN,NaN,1.0,73,2,10,2.0,3,NaN,NaN,NaN,NaN,6,17,60,28,1.0,21.0,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,1.0,3,3,23,9,32,5,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,58,30,9.0,1.0,2.0,NaN,45,69,29,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,6,2.0,9,6.0,1.0,8,10,3,5.0,11.0,56,12,12,10,NaN,2.0,1.0,4,2.0,43,NaN,5,25,61,13,1,1.0,NaN,8,3.0,1.0,NaN,1.0,3.0,9.0,1,NaN,NaN,2.0,1.0,1.0,3.0,NaN,1.0,5.0,NaN,3.0,1.0,4,1.0,1.0,1.0,6,NaN,4.0,NaN,33,1,11,2,1,4.0,1,NaN,NaN,NaN,1.0,19.0,3.0,2,NaN,NaN,3,3,26,9,4,4,1.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,4.0,4,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,8,10.0,3.0,NaN,NaN,1.0,1.0,NaN,17,13,2.0,8,70,2,37,38,52,1.0,NaN,3,NaN,1.0,2,3.0,1.0,5,2.0,2,43,4.0,3,17,23,4,2.0,NaN,4.0,9.0,5,22,1.0,114,NaN,96,NaN,173,6,NaN,769,NaN,1302,3.0,207,85,21,494,340,109,8,123,1156,51,NaN,283,440,NaN,679,95,782,372,NaN,5.0,236,1051,17.0,424,1746,NaN,904,825,477,NaN,474,65,129,1243,178,1478,32,94,428,45,313,309,148,528,1237,26,6,157,69,69,77,90,1185,998,616,259,1525,515,1027,189,16,77,196,47,610,282,475,12,891,NaN,308,2.0,3,715,237,420,145,9.0,711,528,204,58,180,433,44,19,13,2.0,142,NaN,125,96,20,5,NaN,8,5.0,1.0,24
Sour,26,5,16,71,8,4,6,15,10,16,NaN,8,8,5,6,4,2,3,NaN,7,5,5,2,3,1,5,NaN,8,1,NaN,4,2,7,1,1,1,7,9,1,NaN,2,2,1,2,4,1.0,3,4,3,8,5,15,NaN,1,2,3,3,1,2,2,10,7,NaN,1,2,2,7,NaN,6,2,4,3,NaN,1,1,6,3,NaN,NaN,NaN,NaN,1,4.0,11,2,1,3,2,NaN,1,NaN,NaN,2,5,2,NaN,3,2,19,NaN,1,NaN,1,1,NaN,NaN,NaN,1.0,1.0,NaN,NaN

In [118]:
aroma_categorized

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
Sweet Aroma,11,30,99,301,59,14,37,15.0,51,12.0,3.0,50,40,11,13.0,6.0,23,18,15.0,69,35,3,5.0,9,3.0,11.0,1.0,19,1.0,1.0,11.0,23.0,6.0,4.0,3.0,2.0,34,61,12.0,4.0,1.0,7.0,2.0,9,7.0,1.0,12.0,1.0,3.0,33,38.0,28,1.0,1.0,30.0,27,12.0,4.0,15.0,1.0,28.0,4.0,NaN,11,11.0,NaN,41,2.0,60.0,1.0,6.0,6.0,15,1.0,5,13.0,1.0,4.0,1.0,3.0,2.0,2.0,4.0,12.0,23.0,1.0,24.0,23.0,5.0,3.0,8.0,2.0,21.0,5,10.0,2.0,7.0,9.0,77,4.0,3.0,2.0,3.0,2.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,6.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0,2.0,2.0,NaN,1.0,4.0,NaN,4.0,1.0,5.0,1.0,NaN,2.0,NaN,24,28,1.0,1.0,NaN,1.0,4.0,NaN,NaN,46,4.0,2.0,3.0,5.0,1.0,2.0,5.0,5.0,15.0,8.0,NaN,3.0,2.0,NaN,NaN,99,14,NaN,6.0,7.0,1.0,4.0,1.0,NaN,NaN,NaN,15.0,NaN,11,2.0,1.0,1.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,79,NaN,8.0,1.0,3.0,NaN,NaN,NaN,NaN,4.0,11.0,50,26.0,1.0,19.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,1.0,21.0,10.0,19,7.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,46,29,5.0,1.0,1.0,NaN,23.0,49,28,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,5.0,2.0,8.0,6.0,1.0,7.0,7.0,3.0,2.0,10,60.0,11.0,7.0,6.0,NaN,1.0,1.0,2.0,2.0,35,NaN,2.0,14,38,10.0,NaN,1.0,NaN,8.0,NaN,1.0,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,1.0,1.0,3.0,NaN,1.0,2.0,NaN,3.0,1.0,2.0,1.0,NaN,NaN,3.0,NaN,3.0,NaN,30.0,1.0,7.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,1.0,14.0,2.0,1.0,NaN,NaN,1.0,NaN,9,7.0,4.0,3.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,7.0,8.0,2.0,NaN,NaN,NaN,NaN,NaN,17.0,10.0,2.0,8.0,53,2.0,21.0,26.0,46,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,2.0,NaN,41,2.0,2.0,16.0,18,2.0,1.0,NaN,4.0,3.0,3.0,18,1.0,72,NaN,68,NaN,93,3.0,NaN,410,NaN,1209,2.0,132,40.0,17,319,207,87,6.0,98,637,26,NaN,145,303,NaN,457,66,537,178,NaN,4.0,180,666,9,215,1217,NaN,746,648,277,NaN,434,41,80,1000,119,932,24.0,63,255,25,195,204,121,543,979,19,6.0,98,30,55,64,51,657,507,320,187,1185,270,783,163,14,49,122,47,352,187,262,3.0,598,NaN,171,2.0,1.0,324,91,201,101,5.0,347,229,121,43,132,239,30,12,7,1.0,111,NaN,60,71,9.0,5.0,NaN,6,2.0,NaN,10.0
Burnt,8,3,28,78,62,2,11,7.0,15,7.0,1

In [119]:
taste_categorized.shape

(4, 513)

In [120]:
aroma_categorized.shape

(13, 513)

In [11]:
def compute_percentage(x):
    return round(x/x.sum() * 100,1)

In [121]:
taste_percentage = taste_categorized[:-1]
aroma_percentage = aroma_categorized[:-1]

taste_percentage = taste_percentage.apply(compute_percentage,axis=0)
aroma_percentage = aroma_percentage.apply(compute_percentage,axis=0)

In [122]:
taste_percentage

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
Sweet Taste,36.8,43.2,45.1,43.7,44.0,37.7,45.2,38.2,43.8,35.0,40.0,47.4,51.3,38.2,42.6,38.1,48.3,44.9,48.7,45.3,43.2,40.0,41.7,39.0,50.0,38.7,50.0,45.1,42.9,50.0,43.9,46.0,43.8,43.8,41.7,37.5,46.2,44.9,48.3,50.0,20.0,37.5,40.0,45.8,50.0,50.0,41.0,37.5,40.0,43.9,44.3,52.2,42.9,20.0,49.3,45.6,43.8,54.5,46.7,37.5,42.3,46.4,NaN,43.8,54.5,25.0,51.8,55.6,58.3,37.5,45.5,40.0,51.2,33.3,45.5,63.8,20.0,45.5,50.0,46.2,38.5,45.5,55.6,46.4,50.0,42.9,53.2,47.3,47.1,42.9,57.1,42.9,46.9,43.5,43.3,40.0,41.4,42.3,44.3,46.2,45.5,42.9,30.0,33.3,50.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,50.0,43.8,NaN,37.5,50.0,50.0,NaN,50.0,50.0,NaN,NaN,NaN,NaN,NaN,50.0,42.9,45.5,50.0,NaN,50.0,38.5,NaN,36.4,50.0,42.9,42.9,NaN,50.0,NaN,41.3,47.5,50.0,50.0,NaN,50.0,38.9,NaN,NaN,52.9,50.0,57.1,45.5,47.1,50.0,42.9,55.6,46.2,43.3,33.3,50.0,60.0,50.0,50.0,NaN,45.1,37.7,NaN,50.0,33.3,50.0,41.2,36.4,NaN,100.0,NaN,38.2,NaN,36.8,55.6,33.3,36.4,40.0,44.4,NaN,NaN,NaN,NaN,NaN,NaN,50.0,51.0,40.0,52.6,50.0,37.5,NaN,NaN,NaN,NaN,40.0,51.5,61.2,47.5,50.0,46.7,50.0,33.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,100.0,37.5,60.0,52.3,50.0,45.1,45.5,40.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,46.0,53.6,56.2,50.0,66.7,NaN,48.9,50.0,47.5,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,37.5,40.0,50.0,69.2,85.7,50.0,53.3,45.5,30.0,55.6,42.3,58.3,54.5,42.9,41.7,NaN,50.0,33.3,44.4,50.0,49.4,NaN,38.5,44.6,45.9,48.1,25.0,50.0,NaN,42.1,50.0,50.0,NaN,50.0,50.0,47.4,33.3,NaN,NaN,28.6,50.0,50.0,50.0,NaN,50.0,50.0,NaN,75.0,50.0,57.1,50.0,50.0,33.3,46.2,NaN,50.0,NaN,49.3,33.3,55.0,40.0,33.3,50.0,33.3,NaN,NaN,NaN,50.0,46.3,50.0,40.0,NaN,NaN,60.0,33.3,41.9,52.9,44.4,40.0,50.0,42.9,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,66.7,28.6,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,53.3,47.6,50.0,NaN,NaN,50.0,100.0,NaN,48.6,68.4,50.0,44.4,47.6,50.0,50.0,46.9,47.7,50.0,NaN,42.9,NaN,100.0,40.0,60.0,50.0,50.0,50.0,33.3,51.8,66.7,37.5,65.4,43.4,57.1,50.0,NaN,57.1,47.4,41.7,44.9,50.0,53.8,NaN,46.2,NaN,45.3,42.9,NaN,48.0,NaN,57.1,50.0,48.9,48.9,51.2,54.6,54.1,59.2,50.0,55.9,48.7,50.5,NaN,57.9,51.7,NaN,56.2,51.1

In [123]:
aroma_percentage

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512
Sweet Aroma,9.2,15.9,13.3,13.8,17.2,10.5,16.4,10.4,15.7,16.7,16.7,17.5,17.0,12.6,11.6,13.0,14.2,15.3,18.1,20.1,20.3,7.9,11.4,10.6,11.5,15.5,7.7,12.8,10.0,14.3,10.1,17.0,22.2,12.9,10.0,12.5,16.0,13.4,18.5,22.2,11.1,9.5,28.6,17.6,17.1,50.0,14.1,6.7,13.6,14.6,14.7,8.6,5.9,9.1,18.6,17.4,17.6,17.4,21.1,11.1,20.0,8.5,NaN,8.3,16.2,NaN,18.4,12.5,26.8,5.3,10.3,20.7,13.0,16.7,9.3,10.0,9.1,16.0,14.3,9.7,6.7,13.3,23.5,19.0,16.5,5.3,18.0,21.1,10.2,20.0,19.5,10.0,24.7,7.8,16.7,20.0,11.9,20.0,15.1,11.8,13.0,11.1,12.5,11.1,NaN,25.0,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.2,NaN,22.2,NaN,8.3,14.3,50.0,NaN,33.3,16.7,NaN,NaN,NaN,NaN,NaN,25.0,21.4,10.5,13.3,NaN,20.0,12.1,NaN,23.5,12.5,19.2,4.5,NaN,16.7,NaN,14.5,16.5,20.0,20.0,NaN,4.3,9.8,NaN,NaN,16.2,25.0,8.3,10.3,10.2,12.5,12.5,27.8,21.7,20.5,14.5,NaN,20.0,11.1,NaN,NaN,13.3,10.5,NaN,12.0,25.9,20.0,7.7,4.3,NaN,NaN,NaN,10.4,NaN,9.2,9.1,16.7,5.9,13.0,15.2,NaN,NaN,NaN,NaN,NaN,NaN,12.5,23.5,NaN,17.4,10.0,12.5,NaN,NaN,NaN,NaN,13.3,15.7,15.5,16.5,16.7,17.9,10.0,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,10.4,NaN,10.0,6.7,22.1,25.6,12.8,18.4,22.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7,16.8,19.3,12.5,11.1,9.1,NaN,14.5,19.4,18.3,11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,10.0,13.9,18.2,22.9,27.3,12.5,25.0,15.9,13.0,9.1,14.3,26.8,16.2,10.9,11.8,NaN,11.1,12.5,11.1,18.2,18.8,NaN,7.4,10.8,13.1,19.6,NaN,16.7,NaN,17.8,NaN,12.5,NaN,NaN,14.3,10.8,NaN,NaN,NaN,NaN,25.0,12.5,21.4,NaN,11.1,8.3,NaN,15.8,20.0,22.2,50.0,NaN,NaN,12.5,NaN,18.8,NaN,18.6,25.0,18.4,10.0,14.3,18.8,NaN,NaN,NaN,NaN,12.5,17.9,15.4,7.7,NaN,NaN,11.1,NaN,6.9,11.1,17.4,14.3,20.0,14.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,8.7,NaN,NaN,33.3,NaN,NaN,NaN,NaN,NaN,21.9,16.3,11.8,NaN,NaN,NaN,NaN,NaN,22.4,17.2,16.7,21.1,16.4,25.0,12.3,18.1,19.5,NaN,NaN,5.3,NaN,NaN,12.5,8.3,16.7,4.3,18.2,NaN,18.4,18.2,13.3,28.1,18.2,7.1,14.3,NaN,26.7,7.5,10.0,15.3,16.7,18.1,NaN,16.0,NaN,14.4,14.3,NaN,15.5,NaN,18.5,11.8,17.4,15.2,20.7,18.2,17.5,21.6,24.0,21.5,16.4,14.5,NaN,13.3,18.4,NaN,13.5,17.9,18.4,13.0,NaN,20.0,19.0,13.5,15.3,13.8,15.9,NaN,21.2,19.8,15.8,NaN

In [124]:
taste_percentage.shape

(3, 513)

In [125]:
aroma_percentage.shape

(12, 513)

In [126]:
#taste: pick dominant elements with pct > 33%
def dominant_elements_taste(x):
    return list(x[x.values>30].index)

taste_dom = taste_percentage.apply(dominant_elements_taste,axis=0)
taste_dom

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


0              [Sweet Taste, Sour]
1            [Sweet Taste, Bitter]
2            [Sweet Taste, Bitter]
3            [Sweet Taste, Bitter]
4            [Sweet Taste, Bitter]
5            [Sweet Taste, Bitter]
6            [Sweet Taste, Bitter]
7            [Sweet Taste, Bitter]
8            [Sweet Taste, Bitter]
9              [Sweet Taste, Sour]
10           [Sweet Taste, Bitter]
11           [Sweet Taste, Bitter]
12           [Sweet Taste, Bitter]
13           [Sweet Taste, Bitter]
14           [Sweet Taste, Bitter]
15           [Sweet Taste, Bitter]
16           [Sweet Taste, Bitter]
17           [Sweet Taste, Bitter]
18           [Sweet Taste, Bitter]
19           [Sweet Taste, Bitter]
20           [Sweet Taste, Bitter]
21           [Sweet Taste, Bitter]
22           [Sweet Taste, Bitter]
23           [Sweet Taste, Bitter]
24           [Sweet Taste, Bitter]
25           [Sweet Taste, Bitter]
26           [Sweet Taste, Bitter]
27           [Sweet Taste, Bitter]
28           [Sweet 

In [127]:
#aroma: pick 3 dominant elements
def dominant_elements(x):
    return list(x.nlargest(3).index)

aroma_dom = aroma_percentage.apply(dominant_elements,axis=0)
aroma_dom

0         [Fruity, Vegetal, Sweet Aroma]
1           [Fruity, Sweet Aroma, Woody]
2                 [Fruity, Woody, Grain]
3                 [Fruity, Woody, Grain]
4            [Burnt, Sweet Aroma, Woody]
5           [Fruity, Woody, Sweet Aroma]
6           [Sweet Aroma, Fruity, Grain]
7                [Fruity, Grassy, Woody]
8                 [Fruity, Woody, Grain]
9           [Fruity, Woody, Sweet Aroma]
10         [Fruity, Sweet Aroma, Grassy]
11          [Sweet Aroma, Fruity, Grain]
12      [Sweet Aroma, Alcoholic, Fruity]
13          [Fruity, Woody, Sweet Aroma]
14                [Fruity, Woody, Grain]
15          [Fruity, Grain, Sweet Aroma]
16          [Fruity, Woody, Sweet Aroma]
17          [Fruity, Grain, Sweet Aroma]
18           [Burnt, Sweet Aroma, Grain]
19           [Burnt, Sweet Aroma, Grain]
20           [Burnt, Sweet Aroma, Grain]
21               [Fruity, Floral, Spicy]
22          [Grain, Fruity, Sweet Aroma]
23                [Fruity, Woody, Grain]
24          [Fru

In [129]:
#beeradvo df with categorized taste and aroma
df = beeradvo.copy()
df['dom_taste'] = taste_dom.values
df['dom_aroma'] = aroma_dom.values

In [131]:
df.head()

,mnf,name,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link,dom_taste,dom_aroma
0,four winds brewing co.,nectarous,wild ale,5.50%,123,4.45,['look: 4.5 | smell: 4.75 | taste: 4.25 | feel...,"['4.36', '4.2', '5', '3.4', '3.9', '4.54', '4....",35,"[""Poured from the can into a teku glass. No vi...",https://www.beeradvocate.com/beer/profile/3218...,"[Sweet Taste, Sour]","[Fruity, Vegetal, Sweet Aroma]"
1,driftwood brewery,sartori harvest ipa,ipa - american,7.00%,117,4.4,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.28', '4.3', '4.12', '4.24', '4.41', '4.32'...",42,"[""I've heard great things...now I've tasted th...",https://www.beeradvocate.com/beer/profile/1885...,"[Sweet Taste, Bitter]","[Fruity, Sweet Aroma, Woody]"
2,driftwood brewery,fat tug ipa,ipa - american,7.00%,718,4.27,['look: 3.75 | smell: 4.25 | taste: 4 | feel: ...,"['4.05', '4.31', '4.81', '4.17', '3.94', '3.88...",184,['Superb lacing & pours golden pale in the gla...,https://www.beeradvocate.com/beer/profile/1885...,"[Sweet Taste, Bitter]","[Fruity, Woody, Grain]"
3,central city brewers + distillers,red racer ipa (india pale ale),ipa - american,6.50%,1136,4.25,['look: 4 | smell: 3 | taste: 4 | feel: 3 | ov...,"['3.46', '4.21', '3.86', '4.75', '4.5', '3.71'...",499,['Look: Clear dark copper color. Pours a light...,https://www.beeradvocate.com/beer/profile/1032...,"[Sweet Taste, Bitter]","[Fruity, Woody, Grain]"
4,driftwood brewery,singularity,stout - russian imperial,10.60%,180,4.28,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.43', '4.87', '3.2', '4.1', '3.83', '4.21',...",70,"[""2oz bottle (2012, label says 14.0% ABV as op...",https://www.beeradvocate.com/beer/profile/1885...,"[Sweet Taste, Bitter]","[Burnt, Sweet Aroma, Woody]"


In [132]:
#df.to_csv('df_taste_aroma.csv')

In [133]:
df = pd.read_csv("df_taste_aroma.csv").drop("Unnamed: 0", axis=1)

In [134]:
df.head()

,mnf,name,style,alc,ratings,avg_rating,detail_rating,avg_per_user,number_of_reviews,text_reviews,rev_link,dom_taste,dom_aroma
0,four winds brewing co.,nectarous,wild ale,5.50%,123,4.45,['look: 4.5 | smell: 4.75 | taste: 4.25 | feel...,"['4.36', '4.2', '5', '3.4', '3.9', '4.54', '4....",35,"[""Poured from the can into a teku glass. No vi...",https://www.beeradvocate.com/beer/profile/3218...,"['Sweet Taste', 'Sour']","['Fruity', 'Vegetal', 'Sweet Aroma']"
1,driftwood brewery,sartori harvest ipa,ipa - american,7.00%,117,4.4,['look: 4.25 | smell: 4.25 | taste: 4.25 | fee...,"['4.28', '4.3', '4.12', '4.24', '4.41', '4.32'...",42,"[""I've heard great things...now I've tasted th...",https://www.beeradvocate.com/beer/profile/1885...,"['Sweet Taste', 'Bitter']","['Fruity', 'Sweet Aroma', 'Woody']"
2,driftwood brewery,fat tug ipa,ipa - american,7.00%,718,4.27,['look: 3.75 | smell: 4.25 | taste: 4 | feel: ...,"['4.05', '4.31', '4.81', '4.17', '3.94', '3.88...",184,['Superb lacing & pours golden pale in the gla...,https://www.beeradvocate.com/beer/profile/1885...,"['Sweet Taste', 'Bitter']","['Fruity', 'Woody', 'Grain']"
3,central city brewers + distillers,red racer ipa (india pale ale),ipa - american,6.50%,1136,4.25,['look: 4 | smell: 3 | taste: 4 | feel: 3 | ov...,"['3.46', '4.21', '3.86', '4.75', '4.5', '3.71'...",499,['Look: Clear dark copper color. Pours a light...,https://www.beeradvocate.com/beer/profile/1032...,"['Sweet Taste', 'Bitter']","['Fruity', 'Woody', 'Grain']"
4,driftwood brewery,singularity,stout - russian imperial,10.60%,180,4.28,['look: 4.25 | smell: 4.25 | taste: 4.5 | feel...,"['4.43', '4.87', '3.2', '4.1', '3.83', '4.21',...",70,"[""2oz bottle (2012, label says 14.0% ABV as op...",https://www.beeradvocate.com/beer/profile/1885...,"['Sweet Taste', 'Bitter']","['Burnt', 'Sweet Aroma', 'Woody']"


In [ ]:
#reference: https://medium.com/analytics-vidhya/simple-online-review-text-analytics-for-beginners-using-python-c78d34927644

#define elements for evaluation
aroma = "|".join(('aroma'))
flavour = "|".join(('flavour','taste'))

#put those elements in their own list of objects for easy iteration
elements = [aroma,flavour]

#this will create new columns that tell us if an element is in the comment (True or False)
for element in elements:
    review_df[element[0:3]] =review_df['text'].str.contains(element)
#rename the columns for easy use
review_df.rename(columns = {'pil': 'bed', 'cha': 'desk', 'car': 'room', 'tow': 'bathroom'}, inplace = True)

In [ ]:
#name the punctuation you want to clean out of your comments
punctuation = ',?!.\/#@"><[]'
#create a function that splits, lower-cases, and removes punctuation
#The output of this function is a list of all of the above words that are in your review
def room_list(x):
    list_of_words = x.split(" ")
    out_data = []
    for word in list_of_words:
        word = word.lower()
        if word.strip(punctuation) in room_elements:
            out_data.append(word.strip(punctuation))
    return (str(out_data))
#apply that function to our dataframe
review_df['room_list'] = review_df.text.apply(room_list)
review_df.room_list.value_counts()